In [2]:
import pandas as pd
import sqlite3

In [3]:
xlss = pd.ExcelFile('техническое задание (1).xlsx')

In [4]:
#собирает названия листов с 'Таб_'
sheet_names = [name for name in xlss.sheet_names if name.startswith('Таб_')]

In [5]:
#словарь для хранения DataFrame'ов
dfs = {}

In [6]:
#записал датафреймы в словарь
for num, sheet in enumerate(sheet_names):
    dfs[num] = pd.read_excel(xlss,sheet_name=f'{sheet}', header=None)

In [7]:
#удалил лишние столбцы
for k, v in dfs.items():
    v.drop(labels=[9, 10, 11], axis=1, inplace=True)
    

In [8]:
#удалил лишние строки вначале и строку в конце
for k, v in dfs.items():
    v.drop(index=[0,1,2,101], inplace=True)

In [9]:
#сбросил индекс
for k, v in dfs.items():
    v.reset_index(drop=True,inplace=True)

In [10]:
#заполнил строку с месяцами значениями из предыдущей колонки
for k, v in dfs.items():
    v.loc[0].fillna(method='ffill',axis=0,inplace=True)

C:\Users\dimog\AppData\Local\Temp\ipykernel_16584\3409103984.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  v.loc[0].fillna(method='ffill',axis=0,inplace=True)


In [11]:
# получил данные для преобразования
month_year =[]
regions=[]
data=[]
pop=[]
population_types=[]
for k, v in dfs.items():
    month_year.append(v.iloc[0, 1:].values.tolist())
    regions.append(v.iloc[2:, 0].values.tolist())
    data.append(v.iloc[2:, 1:].values.tolist())

    pop.append(v.iloc[1, 1:].values.tolist())
    
    #избавился от лишних пробелов в типе поселения
    population_types.append([' '.join(x.split()) for x in pop[k]])


In [12]:
#исправил название столбца
population_types[0][4] = 'Городское население'

In [13]:
# убрал лишние строки из начала фрейма и обновил индекс
for k, v in dfs.items():
    v.drop(index=[0,1], inplace=True)
    v.reset_index(drop=True,inplace=True)

In [14]:
# словарь для преобразования названия месяца в число
month_dict = {
        'январь': 1,
        'февраль': 2,
        'март': 3,
        'апрель': 4,
        'май': 5,
        'июнь': 6,
        'июль': 7,
        'август': 8,
        'сентябрь': 9,
        'октябрь': 10,
        'ноябрь': 11,
        'декабрь': 12
    }

In [15]:
#преобразование данных и загрузка в новый датафрейм

df_res = pd.DataFrame(columns=[
    'регион',
    'период', 
    'тип поселения',
    'значение показателя', 
    'год', 
    'месяц'])

#цикл по DataFrame'мам каждого листа
for key, value in dfs.items():
    #цикл по самому DataFrame
    for num, lst in enumerate(data[key]):
        #цикл по загрузке данных из каждой строки DataFrame
        for i, v in enumerate(lst):
            tmp = month_year[key][i].split()

            # если значение пропущено или Nan, заменим на 0
            if not isinstance(v, float):
                v = 0
            #изменяет 'январь 2022' на 1/1/2022
            tmp_date = f'{month_dict[tmp[0]]}/1/{tmp[1]}'
            
            region_name = value.iloc[num][0].strip()
            temp = {
                'регион': [region_name],
                'период': [tmp_date],
                'тип поселения': [population_types[key][i]],
                'значение показателя': [v],
                'год': [tmp[1]],
                'месяц': [tmp[0].capitalize()]}
            temp_df = pd.DataFrame(temp)
            df_res = pd.concat([df_res, temp_df], ignore_index=True)

C:\Users\dimog\AppData\Local\Temp\ipykernel_16584\4158762248.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_res = pd.concat([df_res, temp_df], ignore_index=True)


In [16]:
#заменил Nan в 'значения показателя' на 0
df_res['значение показателя'].fillna(0, inplace=True)

In [17]:
#с помощью контекстного менеджера подключился к бд. создал таблицу и загрузил DataFrame

with sqlite3.connect('itog_db.db') as conn:
    df_res.to_sql('table1', conn, if_exists='replace', index=False)

In [18]:
# запрос в бд
with sqlite3.connect('itog_db.db') as conn:
    query = """
    SELECT "год", "тип поселения", ROUND(SUM("значение показателя"), 3)
    FROM table1
    GROUP BY "год", "тип поселения"
"""
df_query = pd.read_sql_query(query, conn)

In [19]:
df_query

,год,тип поселения,"ROUND(SUM(""значение показателя""), 3)"
0,2022,Все население,1731.18
1,2022,Городское население,-272.58
2,2022,Сельское население,287.33
3,2023,Все население,594.60
4,2023,Городское население,-1127.20
5,2023,Сельское население,109.80
6,2024,Все население,-774.60
7,2024,Сельское население,22.60


In [20]:
# загрузил датафрейм с запросом в csv файл
df_query.to_csv('itog_csv.csv')

In [21]:
df=pd.read_csv('itog_csv.csv')

In [22]:
df

,Unnamed: 0,год,тип поселения,"ROUND(SUM(""значение показателя""), 3)"
0,0,2022,Все население,1731.18
1,1,2022,Городское население,-272.58
2,2,2022,Сельское население,287.33
3,3,2023,Все население,594.60
4,4,2023,Городское население,-1127.20
5,5,2023,Сельское население,109.80
6,6,2024,Все население,-774.60
7,7,2024,Сельское население,22.60
